<a href="https://colab.research.google.com/github/Rudra-prasad-tarai/CreditRiskOptimisation/blob/main/Credit_Risk_Optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Rudra-prasad-tarai/CreditRiskOptimisation
!pip install odfpy

Cloning into 'CreditRiskOptimisation'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 20 (delta 5), reused 9 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 29.00 KiB | 9.67 MiB/s, done.
Resolving deltas: 100% (5/5), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160673 sha256=e4b9ae78d6a173d1e8fe2398f3dcd25d5802bfe2f7d0165c1f0a862a3ad1ba42
  Stored in directory: /root/.cache/pip/wheels/d6/1d/c8/8c29be1d73ca42d15977c75193d9f39a98499413c2838ac54c
Successfully built odfpy


In [21]:
import pandas as pd
import numpy  as np

In [134]:
transition_matrix = pd.read_excel('/content/CreditRiskOptimisation/data/CrisilTransitionMatrix.ods',engine= 'odf')

In [135]:
bonds = pd.read_csv('/content/CreditRiskOptimisation/data/MW-Bonds-on-CM-18-Apr-2025.csv')


In [136]:
bonds['CREDIT RATING \n']

,CREDIT RATING \n
0,CRISIL AAA STABLE / CARE AAA / ICRA AAA / IND AAA
1,CRISIL AAA STABLE / CARE AAA / ICRA AAA
2,CRISIL AA Stable / BWR AA+ Negative
3,CRISIL AAA STABLE / CARE AAA / ICRA AAA
4,CRISIL AA Negative / BWR AA+ Negative
...,...
393,NaN
394,NaN
395,NaN
396,IND AAA Stable / CRISIL AAA Stable


In [137]:
bonds.isnull().sum()

,0
SYMBOL \n,0
SERIES \n,15
BOND TYPE \n,118
COUPON RATE \n,0
FACE VALUE \n,0
LTP \n,0
%CHNG \n,0
VOLUME \n(Shares),0
VALUE \n (₹ Crores),0
\n,398


In [138]:
bonds

,SYMBOL \n,SERIES \n,BOND TYPE \n,COUPON RATE \n,FACE VALUE \n,LTP \n,%CHNG \n,VOLUME \n(Shares),VALUE \n (₹ Crores),\n,CREDIT RATING \n,MATURITY DATE \n
0,76NHAI31,NaN,Regular,7.60,"1,000.00","1,100.81",-0.33,"5,678",0.63,NaN,CRISIL AAA STABLE / CARE AAA / ICRA AAA / IND AAA,11-Jan-2031
1,728IRFC30,NG,Regular,7.28,"1,000.00","1,115.00",0.81,"4,469",0.50,NaN,CRISIL AAA STABLE / CARE AAA / ICRA AAA,21-Dec-2030
2,96IIFL28,N5,Regular,9.60,"1,000.00",952.00,-0.05,"4,008",0.38,NaN,CRISIL AA Stable / BWR AA+ Negative,03-Nov-2028
3,734IRFC28,N4,Regular,7.34,"1,000.00","1,078.00",-,"2,515",0.27,NaN,CRISIL AAA STABLE / CARE AAA / ICRA AAA,19-Feb-2028
4,96IIFL28A,NF,Regular,9.60,"1,000.00",956.00,-0.02,"2,760",0.26,NaN,CRISIL AA Negative / BWR AA+ Negative,24-Jun-2028
...,...,...,...,...,...,...,...,...,...,...,...,...
393,925SCL26A,YN,NaN,-,-,-,-,-,-,NaN,NaN,-
394,78ABCL33,N1,NaN,-,-,-,-,-,-,NaN,NaN,-
395,770NABAR27,N4,NaN,-,-,-,-,-,-,NaN,NaN,-
396,795IGT31,NI,Regular,7.95,"1,000.00",-,-,-,-,NaN,IND AAA Stable / CRISIL AAA Stable,06-May-2031


In [139]:
del bonds[bonds.columns[8]]
bonds.dropna(subset = ['CREDIT RATING \n'],inplace=True)

In [140]:

import re

# Extract ONLY CRISIL ratings (case-insensitive, handles variations)
bonds['CREDIT RATING \n'] = bonds['CREDIT RATING \n'].str.extract(r'(CRISIL\s*[A-Za-z+-]+\s*[A-Za-z]*)', flags=re.IGNORECASE)

# Set non-CRISIL entries to NaN
bonds.loc[~bonds['CREDIT RATING \n' ].str.contains('CRISIL', na=False, case=False), 'CREDIT RATING'] = pd.NA

In [141]:
from google.colab import files
df = bonds['CREDIT RATING \n']
df.to_csv('data.csv')
files.download('data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
transition_matrix.head()

,Rating Category,Issuer-months,AAA,AA,A,BBB,BB,B,C,D
0,AAA,15796.0,98.83,1.16,0.01,0.00,0.00,0.00,0.00,0.00
1,AA,40980.0,2.26,96.10,1.53,0.04,0.01,0.00,0.00,0.05
2,A,78111.0,0.14,3.39,93.13,3.13,0.11,0.02,0.01,0.07
3,BBB,211375.0,0.00,0.04,3.20,91.50,4.61,0.11,0.03,0.52
4,BB,308532.0,0.00,0.00,0.01,4.14,88.97,3.73,0.15,3.01
